# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Лабораторная работа 7

In [2]:
%pip install pymorphy2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize

recipes_df = pd.read_csv('preprocessed_descriptions.csv')
words = []
for description in recipes_df['description']:
    words += word_tokenize(description)

words = set(words)

print(words)


1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [10]:
import random
from nltk.metrics import *

words = ['стул', 'столик', 'автомобиль', 'банан', 'дерево', 'дом', 'книга', 'ноутбук', 'молоко', 'шарф']

for i in range(5):
    word1, word2 = random.sample(words, 2)
    distance = edit_distance(word1, word2)
    print(f'Расстояние редактирования между "{word1}" и "{word2}" равно {distance}')


Расстояние редактирования между "ноутбук" и "банан" равно 7
Расстояние редактирования между "ноутбук" и "автомобиль" равно 8
Расстояние редактирования между "стул" и "автомобиль" равно 8
Расстояние редактирования между "банан" и "шарф" равно 4
Расстояние редактирования между "дерево" и "автомобиль" равно 9


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [11]:
def get_closest_words(word, words, k):
    distances = {w: edit_distance(word, w) for w in words}
    r = sorted(distances, key=distances.get)[:k]
    return r

print(get_closest_words('дерево', words, 3))


['дерево', 'дом', 'молоко']


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [ ]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

df = pd.DataFrame(index=word, columns=['word', 'stemmed_word', 'normalized_word'])

for word in words:
    df.at[word, 'word'] = word
    df.at[word, 'stemmed_word'] = stemmer.stem(word)
    df.at[word, 'normalized_word'] = lemmatizer.lemmatize(word)

print(df.head())

2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [14]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print(stop_words)
filtered_words = [w for w in words if w.lower() not in stop_words]

top10_before = pd.Series(words).value_counts().head(10)
top10_after = pd.Series(filtered_words).value_counts().head(10)

stop_words_count = len([w for w in words if w.lower() in stop_words])
total_words_count = len(words)
stop_words_ratio = stop_words_count / total_words_count

print("Доля стоп-слов в общем количестве слов:"(stop_words_ratio))
print(f"Топ-10 самых часто употребляемых слов до удаления стоп-слов:{top10_before}")
print(f"Топ-10 самых часто употребляемых слов после удаления стоп-слов:{top10_after}")

{'same', 'from', 'you', 'then', 'myself', 'does', 'o', 't', 'this', 'more', 'having', 'these', 'on', 'my', 'which', 'i', 'only', 'hadn', 'above', 'for', "doesn't", 'him', 'because', 'while', 'haven', 'it', 'has', 'themselves', 'will', 's', 'be', 'are', 'isn', 'some', 'can', 'no', 'and', 'with', 'after', "you'd", 'out', "mightn't", 'whom', 'she', "you'll", 'all', 'or', 'hasn', 'off', 'just', "haven't", 'the', 'not', 'its', "should've", 'if', 'such', 'aren', 'why', "hasn't", 'over', 'when', 'below', 'up', 'should', 'mightn', "couldn't", 'shan', "shouldn't", 'both', 'd', 'an', 'of', 'a', 'once', "mustn't", 'those', 'too', 'doing', "needn't", 'most', 'wouldn', 'between', "don't", "isn't", 'at', 'needn', 'where', 'your', "she's", 'their', 'here', 'ma', 'her', 'so', 'wasn', 'don', 'to', 'his', 'being', "you're", 'll', 'yourself', 'himself', "wasn't", 'but', 'in', 'didn', "weren't", 'as', 'they', 'any', 'is', "that'll", 'about', 'than', 'down', 'very', 'now', 'won', 'how', 'mustn', 'had', 'co

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('recipes_sample.csv')
random_recipes = df.sample(n=5)

vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(random_recipes['description'])

k = 0
for i, recipe in random_recipes.iterrows():
    print(f"Рецепт {i+1}: {recipe['name']}")
    print(f"Векторное описание: {vectors[k].toarray()}")
    k += 1


Рецепт 189: 30 minute lemon meringue pie
Векторное описание: [[0.         0.         0.         0.         0.         0.
  0.         0.         0.38898761 0.         0.         0.
  0.         0.         0.         0.48214012 0.         0.
  0.         0.         0.48214012 0.         0.         0.
  0.         0.         0.48214012 0.         0.38898761 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]]
Рецепт 12568: greek mojito
Векторное описание: [[0.         0.         0.         0.         0.40824829 0.40824829
  0.         0.         0.         0.         0.         0.
  0.40824829 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.40824829 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.40824829 0.         0.40824829]]
Рецепт 17469: mexican chilli and cheese dip
Ве

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [46]:
from scipy.spatial.distance import cosine

si = []
for i in range(vectors.shape[0]):
    row = []
    for j in range(vectors.shape[0]):
        sim = 1 - cosine(vectors[i].toarray(), vectors[j].toarray())
        row.append(sim)
    si.append(row)

s_df = pd.DataFrame(si, columns=random_recipes['description'])

print(s_df)


description  for a quick and easy lemon meringue pie  \
0                                           1.000000   
1                                           0.000000   
2                                           0.000000   
3                                           0.000000   
4                                           0.550112   

description  greek version of the cuba deliciousness!  posted for zwt #6...  \
0                                                          0.0                
1                                                          1.0                
2                                                          0.0                
3                                                          0.0                
4                                                          0.0                

description  i first had this in a mexican restaurant in the venetian hotel in las vegas, the night before we got married. we loved it so much we virtually drank it and licked the bowl cle

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

**Наиболее похожими являютя рецепты №1 и №5, их схожесть обуславливается тем, что их 'quick and easy' готовить)**